Hogy scaleljük fel a 80x80-as bioszenzor képeket kb 4000x4000-esre?  
Olyan érzésem van hogy ez fontosabb mint maga a modell mert ha a scsaling jó akkor kb adatot "nyerünk" a képekből és könnyebb dolga lesz a modellnek  
Plusz 3D-s a tehát ez is egy plusz dimenzót ad a scalelésnek. Minden idődimben csak a 2D-s képet scaleljük fel? Vagy a teljes 3D-s tenzort? Esetleg az egészből csinálunk egy 4000x4000-es 2D-s képet? Már ha lehet ilyet csinálni.Ehhez lehet egy külön modell kell.

Plusz az idődimenzió is nagyon eltérő:  
mean: 984.0777777777778  
std: 245.44061186082735  
min: 642  
max: 1418  

maszkok statisztikái (mekkora legyen a standard méret?):  
mean:4158.277777777777  
std: 87.17989418792205  
min: 3566  
max: 4253  

Tehát kb 4000/80=50-szeresére kellene növelni a képeket.

Lehetőségek:
- interpoláció: ez a leggyorsabb, de a legrosszabb minőségű is, nagyon sok fajtája van, pl. nearest, linear, cubic, area, lanczos  
- deep learning: pl. GAN, autoencoder?, de ez a leglassabb, és a legnehezebb de talán jobb eredményt ad  
Az esetleges pipelineba nem lehet olyan rétegeket/modellt belerakni ami scalel? Vagy lehet olyat hogy a 80x80-as képet adjuk be és 4000x4000-est várunk?


(kézi skálázás: ez a leglassabb, de a legjobb minőségű is)


Ötletek:  
A 3Ds adatsorból egy 4000x4000-es 2D-s képet csinálunk, majd simán tanítunk rá egy modellt a meglévő maszkokkal.  
Az átalakítás a kérdéses hogy hogyan nem vesztünk információt és kapjuk a legjobb képet.  
Interpoláció lehetne egy lépés:  
- A 3D-a adatsorból egyből egy 4000x4000-es 2D-s képet csinálunk. Egy lépésben nem hiszem hogy van ilyen interpoláció.  
- A 3D-s adatsorból egyből egy 2Ds képet csinálunk, majd ezt skálázzuk fel 4000x4000-esre  
- Egy vagy pár framet választunk és csak azt interpoláljuk fel 4000x4000-esre  
- Saját ötlet hogy kiválasztunk random 50 framet az értékeket összeadjuk majd szétosztjuk 4000x4000-esre. Így a data augmentation is megvan.

Vagy valamilyen konvolúciós hálóval megpróbálunk a 3D-s adatsorból egyből egy 4000x4000-es 2D-s képet csinálni.  

Vagy csak egy hálót használunk aminek a bemenete 80x80x700 és a kimenete 4000x4000.  

adatok egységesítése - milyen módszer? az összes maszk legyen egyforma ->  
levágjuk akkor nem biztos hogy jól ráillik a képre  
ha scaleljük akkor deformálódhat  

a timeseries egyes képeit felscalelni interpolációval? melyik fajta? ezzel szívesen kísérletezek (esetleg átalakítani meglévő algoritmusokat / újat kitalálni)  
mekkorára? maszk méret az 50x vagy elég kissebb?  
hány időpontot/képet? 1, 50, 500, az összes?  
és akkor erre/ezekre a képekre modellt alkotni  

vagy nem nagyon piszkálni az adatokat hanem nyersen 80x80-asan beadni a modelnek  
ebben az esetben a timeseries egy nagyobb részét/egészét szeretnénk inputként hogy legyen plusz információ a modellnek  
az egészet nem  választhatjuk mindenhol mert van ami kétszer olyan hosszú mint egy másik  

esetleg LSTM-es encoder megoldás?


Milyen transzformációk kellenek?
- Downsampling: 4000x4000 -> 80x80 
- Meg a bioszenzorból egyenletes elozslasztású sorozatokat kivenni, 500-1400 képből X darab
- Upsampling: 80x80 -> 4000x4000???
- 

In [ ]:
https://github.com/mehran66/Interpolation-in-Matlab
https://github.com/Qirias/Python-Image-Upscaler
https://github.com/ShambhawiVarchasva/Image-Scale


In [ ]:
import cv2
import numpy as np

img = cv2.imread('your_image.jpg')
res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)

In [ ]:
"""
It does not make good use of Numpy and, thus, is not fast, especially for large images. If you're only rescaling smaller images, it should be fine. I offer this under Apache or MIT license at the discretion of the user.
"""

import math
import numpy

def resize_linear(image_matrix, new_height:int, new_width:int):
    """Perform a pure-numpy linear-resampled resize of an image."""
    output_image = numpy.zeros((new_height, new_width), dtype=image_matrix.dtype)
    original_height, original_width = image_matrix.shape
    inv_scale_factor_y = original_height/new_height
    inv_scale_factor_x = original_width/new_width

    # This is an ugly serial operation.
    for new_y in range(new_height):
        for new_x in range(new_width):
            # If you had a color image, you could repeat this with all channels here.
            # Find sub-pixels data:
            old_x = new_x * inv_scale_factor_x
            old_y = new_y * inv_scale_factor_y
            x_fraction = old_x - math.floor(old_x)
            y_fraction = old_y - math.floor(old_y)

            # Sample four neighboring pixels:
            left_upper = image_matrix[math.floor(old_y), math.floor(old_x)]
            right_upper = image_matrix[math.floor(old_y), min(image_matrix.shape[1] - 1, math.ceil(old_x))]
            left_lower = image_matrix[min(image_matrix.shape[0] - 1, math.ceil(old_y)), math.floor(old_x)]
            right_lower = image_matrix[min(image_matrix.shape[0] - 1, math.ceil(old_y)), min(image_matrix.shape[1] - 1, math.ceil(old_x))]

            # Interpolate horizontally:
            blend_top = (right_upper * x_fraction) + (left_upper * (1.0 - x_fraction))
            blend_bottom = (right_lower * x_fraction) + (left_lower * (1.0 - x_fraction))
            # Interpolate vertically:
            final_blend = (blend_top * y_fraction) + (blend_bottom * (1.0 - y_fraction))
            output_image[new_y, new_x] = final_blend

    return output_image